In [7]:
desc_folder = './results'
colName_mid = 'Compound Name'

desc_rdkit = True
desc_fps = True
desc_cx = True
desc_custom = True

descType_list = []
if desc_custom:
    descType_list.append('custom')
if desc_rdkit:
    descType_list.append('rdkit')
if desc_fps:
    descType_list.append('fingerprints')
if desc_cx:
    descType_list.append('chemaxon')
descType_list
print(f"\tSelected descriptors types: {descType_list}")

	Selected descriptors types: ['custom', 'rdkit', 'fingerprints', 'chemaxon']


In [8]:
import pandas as pd

## load the expt outcome
dataTable_y = pd.read_csv(f'{desc_folder}/outcome_expt.csv')
print(dataTable_y.shape)
dataTable_y.head(3)

(38, 2)


,Compound Name,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num)
0,KT-0001674,0.298013
1,KT-0001675,0.767547
2,KT-0001676,0.186651


In [9]:
## load all descriptor tables and merge together

dataTable_merged = dataTable_y

for descType in descType_list:
    dataTable = pd.read_csv(f'{desc_folder}/descriptors_{descType}.csv')
    print(f"\tThere are total <{dataTable.shape[1]-1}> {descType} descriptors for <{dataTable.shape[0]}> molecules")
    dataTable_merged = dataTable_merged.merge(right=dataTable, on='Compound Name', how='left')
print(f"\tThe merged data table has <{dataTable_merged.shape[0]}> molecules and <{dataTable_merged.shape[1]-1}> descriptors")

## merge the descriptor data with expt outcome
dataTable_y = pd.read_csv(f'{desc_folder}/outcome_expt.csv')

## get descriptors (X)
Descriptors = dataTable_merged.iloc[:, 1:]
print(Descriptors.shape)
dataTable_merged.head(5)

	There are total <2> custom descriptors for <38> molecules
	There are total <140> rdkit descriptors for <38> molecules
	There are total <2048> fingerprints descriptors for <38> molecules
	There are total <33> chemaxon descriptors for <38> molecules
	The merged data table has <38> molecules and <2224> descriptors
(38, 2224)


,Compound Name,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num),custDesc_ADME MDCK(WT) Permeability;Mean;A to B Recovery (%),custDesc_ADME MDCK(WT) Permeability;Mean;B to A Papp (10^-6 cm/s);(Num),rd_MaxAbsEStateIndex,rd_MaxEStateIndex,rd_MinAbsEStateIndex,rd_MinEStateIndex,rd_qed,rd_SPS,...,cx_topology-analyser_chiralCenterCount,cx_topology-analyser_fsp3,cx_topology-analyser_fusedAliphaticRingCount,cx_topology-analyser_fusedAromaticRingCount,cx_topology-analyser_fusedRingCount,cx_topology-analyser_heteroRingCount,cx_topology-analyser_heteroAromaticRingCount,cx_topology-analyser_largestRingSize,cx_topology-analyser_largestRingSystemSize,cx_topology-analyser_rotatableBondCount
0,KT-0001674,0.298013,65.396574,1.327664,14.863094,14.863094,0.132763,-0.719197,0.238176,20.085106,...,2,0.484848,0,2,2,4,1,6,2,11
1,KT-0001675,0.767547,66.234937,8.333819,14.856497,14.856497,0.215315,-0.684509,0.229648,20.434783,...,2,0.515152,0,2,2,4,1,6,2,12
2,KT-0001676,0.186651,22.744314,0.054087,13.135317,13.135317,0.065435,-3.567808,0.060439,16.555556,...,1,0.468085,0,2,2,4,2,6,2,17
3,KT-0001685,0.482284,70.056577,1.064458,14.849205,14.849205,0.093236,-0.739197,0.231174,20.085106,...,2,0.468750,0,2,2,4,1,6,2,11
4,KT-0001686,0.642138,21.732834,0.449181,13.115317,13.115317,0.079324,-3.573348,0.062530,16.555556,...,1,0.456522,0,2,2,4,2,6,2,17


In [10]:
Descriptors = dataTable_merged.iloc[:, 2:]
print(Descriptors.shape)
Descriptors.head(3)

(38, 2223)


,custDesc_ADME MDCK(WT) Permeability;Mean;A to B Recovery (%),custDesc_ADME MDCK(WT) Permeability;Mean;B to A Papp (10^-6 cm/s);(Num),rd_MaxAbsEStateIndex,rd_MaxEStateIndex,rd_MinAbsEStateIndex,rd_MinEStateIndex,rd_qed,rd_SPS,rd_MolWt,rd_HeavyAtomMolWt,...,cx_topology-analyser_chiralCenterCount,cx_topology-analyser_fsp3,cx_topology-analyser_fusedAliphaticRingCount,cx_topology-analyser_fusedAromaticRingCount,cx_topology-analyser_fusedRingCount,cx_topology-analyser_heteroRingCount,cx_topology-analyser_heteroAromaticRingCount,cx_topology-analyser_largestRingSize,cx_topology-analyser_largestRingSystemSize,cx_topology-analyser_rotatableBondCount
0,65.396574,1.327664,14.863094,14.863094,0.132763,-0.719197,0.238176,20.085106,650.708,611.396,...,2,0.484848,0,2,2,4,1,6,2,11
1,66.234937,8.333819,14.856497,14.856497,0.215315,-0.684509,0.229648,20.434783,636.725,595.397,...,2,0.515152,0,2,2,4,1,6,2,12
2,22.744314,0.054087,13.135317,13.135317,0.065435,-3.567808,0.060439,16.555556,899.559,840.087,...,1,0.468085,0,2,2,4,2,6,2,17


In [17]:
import numpy as np
np.array(dataTable_merged['rd_MaxEStateIndex'].to_numpy())

array([14.86309434, 14.85649654, 13.13531675, 14.84920545, 13.11531675,
       14.44791399, 14.84260765, 14.19914336, 14.11382836, 14.14578646,
       14.06605044, 14.18197277, 13.4526609 , 14.36197025, 14.31087047,
       14.32219766, 14.46133272, 14.16835087, 13.43541022, 13.26640593,
       13.08899443, 13.08899443, 13.43067653, 13.53574918, 13.51543302,
       13.44571387, 13.46810768, 13.48107954, 13.51264167, 13.50857073,
       13.50857073, 13.19882141, 14.03413061, 14.42525374, 14.24659717,
       14.02812442, 13.41370943, 13.45479343])

In [13]:
def VarianceFilter(Descriptors, threshold=0):
    ## Removing features with no variance
    from sklearn.feature_selection import VarianceThreshold
    variance_filter = VarianceThreshold(threshold=threshold)
    variance_filter.fit_transform(Descriptors)

    ## generate variance table
    desc_list = list(Descriptors.columns)
    variance_list = variance_filter.variances_.tolist()
    variance_dict = {}
    for i in range(len(desc_list)):
        variance_dict[i] = {}
        variance_dict[i]['Descriptor'] = desc_list[i]
        variance_dict[i]['Variance'] = variance_list[i]
        variance_dict[i]['Select'] = 'Yes' if variance_list[i] > threshold else 'No'
    variance_table = pd.DataFrame.from_dict(variance_dict).T
    ## print results
    [count_Sele, count_Drop] = [variance_table['Select'].value_counts().get(label, 0) for label in ['Yes', 'No']]
    print(f"\tIn total <{len(desc_list)}> desc, there are <{count_Sele}> selected, <{count_Drop}> dropped, cutoff is {threshold}.")
    return variance_table

In [14]:
variance_table = VarianceFilter(Descriptors=Descriptors, threshold=0)
variance_table.head(3)

	In total <2223> desc, there are <938> selected, <1285> dropped, cutoff is 0.


,Descriptor,Variance,Select
0,custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,80.682451,Yes
1,custDesc_ADME MDCK(WT) Permeability;Mean;B to ...,13.612443,Yes
2,rd_MaxAbsEStateIndex,0.298926,Yes


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
X, y = load_iris(return_X_y=True)
X.shape
(150, 4)
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape